## Canalização

| N_Bloco   |   Uplink_Inicial |   Uplink_Final |   Downlink_Inicial |   Downlink_Final | Faixa     |
|:----------|-----------------:|---------------:|-------------------:|-----------------:|:----------|
| 1         |            451   |          451.1 |              461   |            461.1 | 451-468   |
| 2         |            451.1 |          451.2 |              461.1 |            461.2 | 451-468   |
| 3         |            451.2 |          451.3 |              461.2 |            461.3 | 451-468   |
| 4         |            451.3 |          451.4 |              461.3 |            461.4 | 451-468   |
| 5         |            451.4 |          451.5 |              461.4 |            461.5 | 451-468   |
| 6         |            451.5 |          451.6 |              461.5 |            461.6 | 451-468   |
| 7         |            451.6 |          451.7 |              461.6 |            461.7 | 451-468   |
| 8         |            451.7 |          451.8 |              461.7 |            461.8 | 451-468   |
| 9         |            451.8 |          451.9 |              461.8 |            461.9 | 451-468   |
| 10        |            451.9 |          452   |              461.9 |            462   | 451-468   |
| 11        |            452   |          452.1 |              462   |            462.1 | 451-468   |
| 12        |            452.1 |          452.2 |              462.1 |            462.2 | 451-468   |
| 13        |            452.2 |          452.3 |              462.2 |            462.3 | 451-468   |
| 14        |            452.3 |          452.4 |              462.3 |            462.4 | 451-468   |
| 15        |            452.4 |          452.5 |              462.4 |            462.5 | 451-468   |
| 16        |            452.5 |          452.6 |              462.5 |            462.6 | 451-468   |
| 17        |            452.6 |          452.7 |              462.6 |            462.7 | 451-468   |
| 18        |            452.7 |          452.8 |              462.7 |            462.8 | 451-468   |
| 19        |            452.8 |          452.9 |              462.8 |            462.9 | 451-468   |
| 20        |            452.9 |          453   |              462.9 |            463   | 451-468   |
| 21        |            453   |          453.1 |              463   |            463.1 | 451-468   |
| 22        |            453.1 |          453.2 |              463.1 |            463.2 | 451-468   |
| 23        |            453.2 |          453.3 |              463.2 |            463.3 | 451-468   |
| 24        |            453.3 |          453.4 |              463.3 |            463.4 | 451-468   |
| 25        |            453.4 |          453.5 |              463.4 |            463.5 | 451-468   |
| 26        |            453.5 |          453.6 |              463.5 |            463.6 | 451-468   |
| 27        |            453.6 |          453.7 |              463.6 |            463.7 | 451-468   |
| 28        |            453.7 |          453.8 |              463.7 |            463.8 | 451-468   |
| 29        |            453.8 |          453.9 |              463.8 |            463.9 | 451-468   |
| 30        |            453.9 |          454   |              463.9 |            464   | 451-468   |
| 31        |            454   |          454.1 |              464   |            464.1 | 451-468   |
| 32        |            454.1 |          454.2 |              464.1 |            464.2 | 451-468   |
| 33        |            454.2 |          454.3 |              464.2 |            464.3 | 451-468   |
| 34        |            454.3 |          454.4 |              464.3 |            464.4 | 451-468   |
| 35        |            454.4 |          454.5 |              464.4 |            464.5 | 451-468   |
| 36        |            454.5 |          454.6 |              464.5 |            464.6 | 451-468   |
| 37        |            454.6 |          454.7 |              464.6 |            464.7 | 451-468   |
| 38        |            454.7 |          454.8 |              464.7 |            464.8 | 451-468   |
| 39        |            454.8 |          454.9 |              464.8 |            464.9 | 451-468   |
| 40        |            454.9 |          455   |              464.9 |            465   | 451-468   |
| 41        |            455   |          455.1 |              465   |            465.1 | 451-468   |
| 42        |            455.1 |          455.2 |              465.1 |            465.2 | 451-468   |
| 43        |            455.2 |          455.3 |              465.2 |            465.3 | 451-468   |
| 44        |            455.3 |          455.4 |              465.3 |            465.4 | 451-468   |
| 45        |            455.4 |          455.5 |              465.4 |            465.5 | 451-468   |
| 46        |            455.5 |          455.6 |              465.5 |            465.6 | 451-468   |
| 47        |            455.6 |          455.7 |              465.6 |            465.7 | 451-468   |
| 48        |            455.7 |          455.8 |              465.7 |            465.8 | 451-468   |
| 49        |            455.8 |          455.9 |              465.8 |            465.9 | 451-468   |
| 50        |            455.9 |          456   |              465.9 |            466   | 451-468   |
| 51        |            456   |          456.1 |              466   |            466.1 | 451-468   |
| 52        |            456.1 |          456.2 |              466.1 |            466.2 | 451-468   |
| 53        |            456.2 |          456.3 |              466.2 |            466.3 | 451-468   |
| 54        |            456.3 |          456.4 |              466.3 |            466.4 | 451-468   |
| 55        |            456.4 |          456.5 |              466.4 |            466.5 | 451-468   |
| 56        |            456.5 |          456.6 |              466.5 |            466.6 | 451-468   |
| 57        |            456.6 |          456.7 |              466.6 |            466.7 | 451-468   |
| 58        |            456.7 |          456.8 |              466.7 |            466.8 | 451-468   |
| 59        |            456.8 |          456.9 |              466.8 |            466.9 | 451-468   |
| 60        |            456.9 |          457   |              466.9 |            467   | 451-468   |
| 61        |            457   |          457.1 |              467   |            467.1 | 451-468   |
| 62        |            457.1 |          457.2 |              467.1 |            467.2 | 451-468   |
| 63        |            457.2 |          457.3 |              467.2 |            467.3 | 451-468   |
| 64        |            457.3 |          457.4 |              467.3 |            467.4 | 451-468   |
| 65        |            457.4 |          457.5 |              467.4 |            467.5 | 451-468   |
| 66        |            457.5 |          457.6 |              467.5 |            467.6 | 451-468   |
| 67        |            457.6 |          457.7 |              467.6 |            467.7 | 451-468   |
| 68        |            457.7 |          457.8 |              467.7 |            467.8 | 451-468   |
| 69        |            457.8 |          457.9 |              467.8 |            467.9 | 451-468   |
| 70        |            457.9 |          458   |              467.9 |            468   | 451-468   |
| 1         |            703   |          708   |              758   |            763   | 703-803   |
| 2         |            708   |          713   |              763   |            768   | 703-803   |
| 3         |            713   |          718   |              768   |            773   | 703-803   |
| 4         |            718   |          723   |              773   |            778   | 703-803   |
| 5         |            723   |          728   |              778   |            783   | 703-803   |
| 6         |            728   |          733   |              783   |            788   | 703-803   |
| 7         |            733   |          738   |              788   |            793   | 703-803   |
| 8         |            738   |          743   |              793   |            798   | 703-803   |
| 9         |            743   |          748   |              798   |            803   | 703-803   |
| 1         |            819   |          824   |              864   |            869   | 819-894   |
| 2         |            824   |          829   |              869   |            874   | 819-894   |
| 3         |            829   |          834   |              874   |            879   | 819-894   |
| 4         |            834   |          839   |              879   |            884   | 819-894   |
| 5         |            839   |          844   |              884   |            889   | 819-894   |
| 6         |            844   |          849   |              889   |            894   | 819-894   |
| 1         |            898.5 |          901   |              943.5 |            946   | 898.5-960 |
| 2         |            905   |          907.5 |              950   |            952.5 | 898.5-960 |
| 3         |            907.5 |          910   |              952.5 |            955   | 898.5-960 |
| 4         |            910   |          912.5 |              955   |            957.5 | 898.5-960 |
| 5         |            912.5 |          915   |              957.5 |            960   | 898.5-960 |
| 1         |           1710   |         1715   |             1805   |           1810   | 1710-1880 |
| 2         |           1715   |         1720   |             1810   |           1815   | 1710-1880 |
| 3         |           1720   |         1725   |             1815   |           1820   | 1710-1880 |
| 4         |           1725   |         1730   |             1820   |           1825   | 1710-1880 |
| 5         |           1730   |         1735   |             1825   |           1830   | 1710-1880 |
| 6         |           1735   |         1740   |             1830   |           1835   | 1710-1880 |
| 7         |           1740   |         1745   |             1835   |           1840   | 1710-1880 |
| 8         |           1745   |         1750   |             1840   |           1845   | 1710-1880 |
| 9         |           1750   |         1755   |             1845   |           1850   | 1710-1880 |
| 10        |           1755   |         1760   |             1850   |           1855   | 1710-1880 |
| 11        |           1760   |         1765   |             1855   |           1860   | 1710-1880 |
| 12        |           1765   |         1770   |             1860   |           1865   | 1710-1880 |
| 13        |           1770   |         1775   |             1865   |           1870   | 1710-1880 |
| 14        |           1775   |         1780   |             1870   |           1875   | 1710-1880 |
| 15        |           1780   |         1785   |             1875   |           1880   | 1710-1880 |
| 1         |           1920   |         1925   |             2110   |           2115   | 1920-2170 |
| 2         |           1925   |         1930   |             2115   |           2120   | 1920-2170 |
| 3         |           1930   |         1935   |             2120   |           2125   | 1920-2170 |
| 4         |           1935   |         1940   |             2125   |           2130   | 1920-2170 |
| 5         |           1940   |         1945   |             2130   |           2135   | 1920-2170 |
| 6         |           1945   |         1950   |             2135   |           2140   | 1920-2170 |
| 7         |           1950   |         1955   |             2140   |           2145   | 1920-2170 |
| 8         |           1955   |         1960   |             2145   |           2150   | 1920-2170 |
| 9         |           1960   |         1965   |             2150   |           2155   | 1920-2170 |
| 10        |           1965   |         1970   |             2155   |           2160   | 1920-2170 |
| 11        |           1970   |         1975   |             2160   |           2165   | 1920-2170 |
| 12        |           1975   |         1980   |             2165   |           2170   | 1920-2170 |
| 1         |           1980   |         1985   |             2170   |           2175   | 1980-2200 |
| 2         |           1985   |         1990   |             2175   |           2180   | 1980-2200 |
| 3         |           1990   |         1995   |             2180   |           2185   | 1980-2200 |
| 4         |           1995   |         2000   |             2185   |           2190   | 1980-2200 |
| 5         |           2000   |         2005   |             2190   |           2195   | 1980-2200 |
| 6         |           2005   |         2010   |             2195   |           2200   | 1980-2200 |
| 1         |           2500   |         2505   |             2620   |           2625   | 2500-2690 |
| 2         |           2505   |         2510   |             2625   |           2630   | 2500-2690 |
| 3         |           2510   |         2515   |             2630   |           2635   | 2500-2690 |
| 4         |           2515   |         2520   |             2635   |           2640   | 2500-2690 |
| 5         |           2520   |         2525   |             2640   |           2645   | 2500-2690 |
| 6         |           2525   |         2530   |             2645   |           2650   | 2500-2690 |
| 7         |           2530   |         2535   |             2650   |           2655   | 2500-2690 |
| 8         |           2535   |         2540   |             2655   |           2660   | 2500-2690 |
| 9         |           2540   |         2545   |             2660   |           2665   | 2500-2690 |
| 10        |           2545   |         2550   |             2665   |           2670   | 2500-2690 |
| 11        |           2550   |         2555   |             2670   |           2675   | 2500-2690 |
| 12        |           2555   |         2560   |             2675   |           2680   | 2500-2690 |
| 13        |           2560   |         2565   |             2680   |           2685   | 2500-2690 |
| 14        |           2565   |         2570   |             2685   |           2690   | 2500-2690 |
| 15        |           2570   |         2575   |             2570   |           2575   | 2500-2690 |
| 16        |           2575   |         2580   |             2575   |           2580   | 2500-2690 |
| 17        |           2580   |         2585   |             2580   |           2585   | 2500-2690 |
| 18        |           2585   |         2590   |             2585   |           2590   | 2500-2690 |
| 19        |           2590   |         2595   |             2590   |           2595   | 2500-2690 |
| 20        |           2595   |         2600   |             2595   |           2600   | 2500-2690 |
| 21        |           2600   |         2605   |             2600   |           2605   | 2500-2690 |
| 22        |           2605   |         2610   |             2605   |           2610   | 2500-2690 |
| 23        |           2610   |         2615   |             2610   |           2615   | 2500-2690 |
| 24        |           2615   |         2620   |             2615   |           2620   | 2500-2690 |
| C-3       |            nan   |          nan   |             2570   |           2576   | 2570-2618 |
| C-4       |            nan   |          nan   |             2582   |           2588   | 2570-2618 |
| D-3       |            nan   |          nan   |             2576   |           2582   | 2570-2618 |
| D-4       |            nan   |          nan   |             2588   |           2594   | 2570-2618 |
| E-1       |            nan   |          nan   |             2594   |           2600   | 2570-2618 |
| E-2       |            nan   |          nan   |             2606   |           2612   | 2570-2618 |
| F-1       |            nan   |          nan   |             2600   |           2606   | 2570-2618 |
| F-2       |            nan   |          nan   |             2612   |           2618   | 2570-2618 |

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

In [ ]:
from extracao.format import parse_bw
from extracao.updates import split_designacao

In [ ]:
COLS_SMP = {
    "NumAto": "Num_Ato",
    "NumFistel": "Fistel",
    "NomeEntidade": "Entidade",
    "SiglaUf": "UF",
    "NumEstacao": "Estação",
    "CodMunicipio": "Código_Município",
    "DataValidade": "Validade_RF",
    "FreqInicialMHz": "Frequência_Inicial",
    "FreqCentralMHz": "Frequência_Central",
    "FreqTxMHz": "Frequência_Transmissão",
    "FreqRxMHz": "Frequência_Recepção",
    "FreqFinalMHz": "Frequência_Final",
    "formId": "Tipo_Estação",
    "Tecnologia": "Tecnologia",
    "Latitude": "Latitude",
    "Longitude": "Longitude",
    "DesignacaoEmissao": "Designação_Emissão",
    "PotenciaTransmissorWatts": "Potência(W)",
    "CodTipoAntena": "Cod_TipoAntena",
    "Polarizacao": "Polarização",
    "RaioAntena": "Raio_Antena",
    "GanhoAntena": "Ganho_Antena",
    "FrenteCostaAntena": "FC_Antena",
    "AnguloMeiaPotenciaAntena": "Ang_MP_Antena",
    "AnguloElevacao": "Ângulo_Elevação",
    "Azimute": "Azimute",
    "AlturaAntena": "Altura_Antena",
    "PerdasAcessorias": "Perdas_Acessorias",
}

DTYPE_SMP = {
    "NumAto": "category",
    "NumFistel": "category",
    "NomeEntidade": "category",
    "SiglaUf": "category",
    "NumEstacao": "string",
    "CodMunicipio": "category",
    "DataValidade": "category",
    "FreqInicialMHz": "category",
    "FreqCentralMHz": "category",
    "FreqFinalMHz": "category",
    "FreqTxMHz": "category",
    "FreqRxMHz": "category",
    "formId": "category",
    "Tecnologia": "category",
    "Latitude": "float",
    "Longitude": "float",
    "DesignacaoEmissao": "category",
    "PotenciaTransmissorWatts": "float",
    "CodTipoAntena": "category",
    "Polarizacao": "category",
    "RaioAntena": "float",
    "GanhoAntena": "float",
    "FrenteCostaAntena": "float",
    "AnguloMeiaPotenciaAntena": "float",
    "AnguloElevacao": "float",
    "Azimute": "float",
    "AlturaAntena": "float",
    "PerdasAcessorias": "float",
}


In [ ]:
folder = Path.cwd().parent / 'dados'
df = pd.read_parquet(folder / 'smp_formated.parquet.gzip')

In [ ]:
df = df.rename(columns=COLS_SMP)

In [ ]:
df = split_designacao(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1946684 entries, 0 to 1946610
Data columns (total 29 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   Num_Ato                 string
 1   Validade_RF             string
 2   Fistel                  string
 3   Entidade                string
 4   Estação          string
 5   Latitude                string
 6   Longitude               string
 7   Frequência_Inicial      string
 8   Frequência_Final        string
 9   Frequência_Central      string
 10  Código_Município        string
 11  UF                      string
 12  Potência(W)             string
 13  Cod_TipoAntena          string
 14  Polarização             string
 15  Raio_Antena             string
 16  Ganho_Antena            string
 17  FC_Antena               string
 18  Ang_MP_Antena           string
 19  Ângulo_Elevação         string
 20  Azimute                 string
 21  Altura_Antena           string
 22  Perdas_Acessorias       string

A Frequência Central possui muitas lacunas nos dados, a frequência de transmissão será o dado utilizado.

In [ ]:
df[df.Frequência_Central.isna()].shape

(591400, 29)

In [ ]:
df[df.Frequência_Central == df.Frequência_Transmissão].shape

(297944, 29)

In [ ]:
df[df.Frequência_Final == df.Frequência_Transmissão].shape

(626866, 29)

In [ ]:
df[df.Frequência_Inicial == df.Frequência_Transmissão].shape

(24, 29)

Criar canais de Transmissão - Downlink

In [ ]:
bw = df['Largura_Emissão(kHz)'].astype('float') / 2000 # Unidade em kHz
df['Início_Canal_Down'] = df.Frequência_Transmissão.astype(float) - bw 
df['Fim_Canal_Down'] = df.Frequência_Transmissão.astype(float) + bw
df['Início_Canal_Up'] = df.Frequência_Recepção.astype(float) - bw 
df['Fim_Canal_Up'] = df.Frequência_Recepção.astype(float) + bw

In [ ]:
df.groupby(['Frequência_Transmissão', 'Frequência_Recepção'], as_index=False).size().sort_values('size', ascending=False)

,Frequência_Transmissão,Frequência_Recepção,size
125,874.5,829.5,92135
80,2160.0,1970.0,87642
99,2680.0,2560.0,69351
114,788.0,733.0,67617
74,2140.0,1950.0,63948
...,...,...,...
104,3640.0,3640.0,3
65,1960.0,2150.0,3
110,778.0,880.0,3
89,25600.0,25600.0,2


Eliminar colunas que não serão utilizadas

In [ ]:
df.drop(['Frequência_Inicial', 'Frequência_Central', 'Frequência_Final'], axis=1, inplace=True)

Como vamos agrupar diversas estações baseadas em um subconjunto de colunas, vamos ordenar os dados de forma crescente pelo número da estação. Assim ao agruparmos diversos registros num só, o número da estação desse agrupamento será a estação licenciada mais antiga do grupo, em outras palavras, será mantido o número da estação menor por conta do ordenamento.

In [ ]:
df['Estação'] = df['Estação'].astype(int) 
df = df.sort_values('Estação', ignore_index=True)
df = df.astype('string', copy=False)

In [ ]:
subset = ['UF', 'Código_Município', 'Fistel', 'Frequência_Transmissão', 'Frequência_Recepção', 'Largura_Emissão(kHz)', 'Classe_Emissão']

É preciso eliminar dados vazios em qualquer das colunas que serão agrupadas, do contrário há um desalinhamento ao tentar juntar a contagem dos dados agregados.

In [ ]:
df.dropna(subset=subset, axis=0, inplace=True)

In [ ]:
df_sub = df[~df.duplicated(subset=subset, keep="first")].reset_index(drop=True)

In [ ]:
df_sub['Multiplicidade'] = df.groupby(subset, sort=False).size().values

In [ ]:
df_sub.sample(10)

,Num_Ato,Validade_RF,Fistel,Entidade,Estação,Latitude,Longitude,Código_Município,UF,Potência(W),...,Tecnologia,Frequência_Transmissão,Frequência_Recepção,Largura_Emissão(kHz),Classe_Emissão,Início_Canal_Down,Fim_Canal_Down,Início_Canal_Up,Fim_Canal_Up,Multiplicidade
115192,689802007,2017-12-13,50409307637,CLARO S.A.,1000853184,-4.37917,-44.66972,2111409,MA,60.0,...,LTE,1835.0,1832.5,10000.0,G7W,1830.0,1840.0,1827.5,1837.5,3
82694,169212022.0,2032-12-22,50409307637,CLARO S.A.,690399260,-2.48671,-44.24434,2111300,MA,60.0,...,LTE,1833.75,1738.75,2500.0,G7W,1832.5,1835.0,1737.5,1740.0,1
99017,35322011,2023-04-30,50409428698,TIM S A,692276025,-0.168676,-49.982015,1502509,PA,60.0,...,GSM,1827.5,1732.5,200.0,G7W,1827.4,1827.6,1732.4,1732.6,3
115090,12392015.0,2023-04-30,50409307637,CLARO S.A.,1000841062,-0.80083,-47.60333,1504109,PA,61.6,...,WCDMA,2170.0,2165.0,5000.0,D9W,2167.5,2172.5,2162.5,2167.5,3
34231,41662008,2024-01-20,50409146447,TELEFÔNICA BRASIL S.A.,641263180,-21.35111,-51.75417,3547106,SP,10.000,...,GSM,880.0,869.0,200.0,G7W,879.9,880.1,868.9,869.1,8
77385,35312011,2023-04-30,50409146366,TELEFONICA BRASIL S.A.,689557728,-25.915528,-49.174889,4127601,PR,46.0,...,LTE,1855.0,1760.0,5000.0,G7W,1852.5,1857.5,1757.5,1762.5,3
87217,76882011,2028-04-01,50409307637,CLARO S.A.,690779380,-20.1498,-40.1859,3205002,ES,80.0,...,LTE,1875.0,1780.0,5000.0,G7W,1872.5,1877.5,1777.5,1782.5,28
123773,48372023,2035-04-19,50409307637,CLARO S.A.,1004757902,-18.74001,-47.5138,3143104,MG,81.85,...,WCDMA,2117.5,1927.5,5000.0,D7W,2115.0,2120.0,1925.0,1930.0,6
100725,35312011,2023-04-30,50409146285,TELEFONICA BRASIL S.A.,692571582,-2.69056,-54.64222,1504752,PA,74.0,...,GSM,1867.5,1772.5,200.0,G7W,1867.4,1867.6,1772.4,1772.6,6
90133,17012008.0,2023-04-30,50409146366,TELEFÔNICA BRASIL S.A.,690984960,-23.46917,-53.08333,4117206,PR,49.100,...,WCDMA,2165.0,2155.0,5000.0,G9W,2162.5,2167.5,2152.5,2157.5,3


In [ ]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143973 entries, 0 to 143972
Data columns (total 31 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Num_Ato                 143973 non-null  string
 1   Validade_RF             143973 non-null  string
 2   Fistel                  143973 non-null  string
 3   Entidade                143973 non-null  string
 4   Estação          143973 non-null  string
 5   Latitude                143973 non-null  string
 6   Longitude               143973 non-null  string
 7   Código_Município        143973 non-null  string
 8   UF                      143973 non-null  string
 9   Potência(W)             143973 non-null  string
 10  Cod_TipoAntena          143973 non-null  string
 11  Polarização             143973 non-null  string
 12  Raio_Antena             55691 non-null   string
 13  Ganho_Antena            143973 non-null  string
 14  FC_Antena               143973 non-null  st

Lendo e formatando o arquivo de canais do serviço SMP, construído à partir da  [Resolução nº 757, de 08 de novembro de 2022](https://informacoes.anatel.gov.br/legislacao/resolucoes/2022/1760-resolucao-757)

In [ ]:
canais = pd.read_excel(folder / 'canalização_smp.xlsx', dtype = 'string')
canais

,N_Bloco,Uplink_Inicial,Uplink_Final,Downlink_Inicial,Downlink_Final,Largura_Bloco(MHz),Offset,Faixa,Tecnologia
0,1,451,451.1,461,461.1,0.10000000000002274,10,451-468,UHF
1,2,451.1,451.2,461.1,461.2,0.0999999999999659,10,451-468,UHF
2,3,451.2,451.3,461.2,461.3,0.10000000000002274,10,451-468,UHF
3,4,451.3,451.4,461.3,461.4,0.0999999999999659,10,451-468,UHF
4,5,451.4,451.5,461.4,461.5,0.10000000000002274,10,451-468,UHF
...,...,...,...,...,...,...,...,...,...
142,20,2595,2600,2595,2600,5,0,2500-2690,GSM|LTE
143,21,2600,2605,2600,2605,5,0,2500-2690,GSM|LTE
144,22,2605,2610,2605,2610,5,0,2500-2690,GSM|LTE
145,23,2610,2615,2610,2615,5,0,2500-2690,GSM|LTE


In [ ]:
canais[['Downlink_Inicial', 'Downlink_Final']] = canais[['Downlink_Inicial', 'Downlink_Final']].astype(float)
canais[['Uplink_Inicial', 'Uplink_Final']] = canais[['Uplink_Inicial', 'Uplink_Final']].astype(float)
canais = canais.sort_values(['Downlink_Inicial'], ignore_index=True)
canais['N_Bloco'] = canais['N_Bloco'].str.strip()
canais['Faixa'] = canais['Faixa'].str.strip()

In [ ]:
df_sub[['Início_Canal_Down', 'Fim_Canal_Down', 'Início_Canal_Up', 'Fim_Canal_Up']] = df_sub[['Início_Canal_Down', 'Fim_Canal_Down', 'Início_Canal_Up', 'Fim_Canal_Up']].astype(float)

In [ ]:
conjunto_de_canais = df_sub.groupby(['Início_Canal_Down', 'Fim_Canal_Down', 'Início_Canal_Up', 'Fim_Canal_Up']).size().to_frame('Total').reset_index()
conjunto_de_canais.sort_values('Total', ascending=False, inplace=True, ignore_index=True)
conjunto_de_canais

,Início_Canal_Down,Fim_Canal_Down,Início_Canal_Up,Fim_Canal_Up,Total
0,872.0000,877.0000,827.0000,832.0000,4863
1,783.0000,793.0000,728.0000,738.0000,4071
2,2157.5000,2162.5000,1967.5000,1972.5000,3810
3,773.0000,783.0000,718.0000,728.0000,3651
4,2137.5000,2142.5000,1947.5000,1952.5000,2991
...,...,...,...,...,...
461,889.2500,890.7500,879.2500,880.7500,1
462,2665.7550,2674.2450,2655.7550,2664.2450,1
463,1838.7500,1841.2500,1743.7500,1746.2500,1
464,957.6500,959.8500,912.6500,914.8500,1


In [ ]:
# from tqdm.auto import tqdm
# for row in tqdm(conjunto_de_canais.itertuples()):
#     for subset in canais.groupby('Faixa'):
#         canal, canais_subset = subset
#         begin = canais_subset[(row.Início_Canal >= canais_subset['Downlink_Inicial']) & (row.Início_Canal < canais_subset['Downlink_Final'])]
#         end = canais_subset[(row.Fim_Canal > canais_subset['Downlink_Inicial']) & (row.Fim_Canal <= canais_subset['Downlink_Final'])]
#         if not begin.empty and not end.empty:
#             blocos = '|'.join(canais_subset.loc[begin.index[0]:end.index[-1], 'N_Bloco'])
#             conjunto_de_canais.loc[row.Index, ['Blocos', 'Faixas']] = blocos, canal
#             break

In [ ]:
# from tqdm.auto import tqdm
# for row in tqdm(conjunto_de_canais.itertuples()):
#     begin = canais[(row.Início_Canal >= canais['Downlink_Inicial']) & (row.Início_Canal < canais['Downlink_Final'])]
#     end = canais[(row.Fim_Canal > canais['Downlink_Inicial']) & (row.Fim_Canal <= canais['Downlink_Final'])]
#     if not begin.empty and not end.empty:
#         blocos = '|'.join(canais.loc[begin.index[0]:end.index[0], 'N_Bloco'])
#         faixas = '|'.join(canais.loc[begin.index[0]:end.index[0], 'Faixa'].unique())
#         conjunto_de_canais.loc[row.Index, ['Blocos', 'Faixas']] = blocos, faixas    

In [ ]:
from tqdm.auto import tqdm
conjunto_de_canais['Intervalo_Canal_Down'] = 'Inválido'
for row in tqdm(conjunto_de_canais.itertuples()):
    interval = canais[(row.Início_Canal_Down < canais['Downlink_Final']) & (row.Fim_Canal_Down > canais['Downlink_Inicial'])]
    faixa = 'Downlink'
    if interval.empty:
        interval = canais[(row.Início_Canal_Down < canais['Uplink_Final']) & (row.Fim_Canal_Down > canais['Uplink_Inicial'])]
        faixa = 'Uplink'
    else:
        down = ' | '.join(interval[['Downlink_Inicial', 'Downlink_Final']].apply(lambda x: f'{x[0]}-{x[1]}', axis=1))
        faixas = ' | '.join(interval.Faixa.unique())
        conjunto_de_canais.loc[row.Index, ['Blocos_Downlink', 'Faixas_Downlink', 'Intervalo_Canal_Down']] = down, faixas, faixa
    

0it [00:00, ?it/s]

In [ ]:
conjunto_de_canais['Intervalo_Canal_Up'] = 'Inválido'
for row in tqdm(conjunto_de_canais.itertuples()):
    interval = canais[(row.Início_Canal_Up < canais['Uplink_Final']) & (row.Fim_Canal_Up > canais['Uplink_Inicial'])]
    faixa = 'Uplink'
    if interval.empty:
        interval = canais[(row.Início_Canal_Up < canais['Downlink_Final']) & (row.Fim_Canal_Up > canais['Downlink_Inicial'])]
        faixa = 'Downlink'
    else:
        up = ' | '.join(interval[['Uplink_Inicial', 'Uplink_Final']].apply(lambda x: f'{x[0]}-{x[1]}', axis=1))
        faixas = ' | '.join(interval.Faixa.unique())
        conjunto_de_canais.loc[row.Index, ['Blocos_Uplink', 'Faixas_Uplink', 'Intervalo_Canal_Up']] = up, faixas, faixa
    

0it [00:00, ?it/s]

In [ ]:
conjunto_de_canais = conjunto_de_canais[['Início_Canal_Down', 'Fim_Canal_Down', 'Blocos_Downlink', 'Faixas_Downlink', 'Intervalo_Canal_Down',
                                         'Início_Canal_Up', 'Fim_Canal_Up', 'Blocos_Uplink', 'Faixas_Uplink', 'Intervalo_Canal_Up', 'Total']]
conjunto_de_canais.to_excel(folder / 'canais2canalização.xlsx', index=False)

In [ ]:
conjunto_de_canais

,Início_Canal_Down,Fim_Canal_Down,Blocos_Downlink,Faixas_Downlink,Intervalo_Canal_Down,Início_Canal_Up,Fim_Canal_Up,Blocos_Uplink,Faixas_Uplink,Intervalo_Canal_Up,Total
0,872.0000,877.0000,869.0-874.0 | 874.0-879.0,819-894,Downlink,827.0000,832.0000,824.0-829.0 | 829.0-834.0,819-894,Uplink,4863
1,783.0000,793.0000,783.0-788.0 | 788.0-793.0,703-803,Downlink,728.0000,738.0000,728.0-733.0 | 733.0-738.0,703-803,Uplink,4071
2,2157.5000,2162.5000,2155.0-2160.0 | 2160.0-2165.0,1920-2170,Downlink,1967.5000,1972.5000,1965.0-1970.0 | 1970.0-1975.0,1920-2170,Uplink,3810
3,773.0000,783.0000,773.0-778.0 | 778.0-783.0,703-803,Downlink,718.0000,728.0000,718.0-723.0 | 723.0-728.0,703-803,Uplink,3651
4,2137.5000,2142.5000,2135.0-2140.0 | 2140.0-2145.0,1920-2170,Downlink,1947.5000,1952.5000,1945.0-1950.0 | 1950.0-1955.0,1920-2170,Uplink,2991
...,...,...,...,...,...,...,...,...,...,...,...
461,889.2500,890.7500,889.0-894.0,819-894,Downlink,879.2500,880.7500,NaN,NaN,Inválido,1
462,2665.7550,2674.2450,2665.0-2670.0 | 2670.0-2675.0,2500-2690,Downlink,2655.7550,2664.2450,NaN,NaN,Inválido,1
463,1838.7500,1841.2500,1835.0-1840.0 | 1840.0-1845.0,1710-1880,Downlink,1743.7500,1746.2500,1740.0-1745.0 | 1745.0-1750.0,1710-1880,Uplink,1
464,957.6500,959.8500,957.5-960.0,898.5-960,Downlink,912.6500,914.8500,912.5-915.0,898.5-960,Uplink,1


In [ ]:
conjunto_de_canais.Intervalo_Canal_Down.value_counts()

Intervalo_Canal_Down
Downlink    430
Inválido     36
Name: count, dtype: int64

In [ ]:
conjunto_de_canais.Intervalo_Canal_Up.value_counts()

Intervalo_Canal_Up
Inválido    240
Uplink      226
Name: count, dtype: int64

In [ ]:
df_sub.sample(5)

,Num_Ato,Validade_RF,Fistel,Entidade,Estação,Latitude,Longitude,Código_Município,UF,Potência(W),...,Tecnologia,Frequência_Transmissão,Frequência_Recepção,Largura_Emissão(kHz),Classe_Emissão,Início_Canal_Down,Fim_Canal_Down,Início_Canal_Up,Fim_Canal_Up,Multiplicidade
13096,48372023,2028-03-31,50409105090,CLARO S.A.,399686924,-22.73103,-47.63716,3538709,SP,60.0,...,LTE,2117.5,1927.5,10000.0,G7W,2112.50,2122.50,1922.50,1932.50,28
134826,29272013,2023-04-30,50409314250,TIM S A,1013053335,-12.087889,-51.4025,5107883,MT,20.0,...,GSM,1831.25,1736.25,200.0,G7W,1831.15,1831.35,1736.15,1736.35,6
9142,97432014,2028-11-29,50409146285,TELEFONICA BRASIL S.A.,327608595,-21.97944,-44.94083,3115508,MG,40.0,...,LTE,788.0,733.0,10000.0,G7W,783.00,793.00,728.00,738.00,6
5759,48412023,2027-10-18,50409146366,TELEFONICA BRASIL S.A.,5157498,-29.815,-53.38917,4315503,RS,78.0,...,WCDMA,2160.0,1970.0,5000.0,G9W,2157.50,2162.50,1967.50,1972.50,3
12495,69272020,2023-04-30,50409428698,TIM S A ...,399655778,-8.116389,-35.298333,2616407,PE,40.0,...,WCDMA,2130.0,1940.0,5000.0,G7W,2127.50,2132.50,1937.50,1942.50,6


In [ ]:
df_out = pd.merge(df_sub, conjunto_de_canais, how='left', on=['Início_Canal_Down', 'Fim_Canal_Down', 'Início_Canal_Up', 'Fim_Canal_Up'])

In [ ]:
df_out.Intervalo_Canal_Down.value_counts()

Intervalo_Canal_Down
Downlink    142571
Inválido      1402
Name: count, dtype: int64

In [ ]:
df_out.Intervalo_Canal_Up.value_counts()

Intervalo_Canal_Up
Uplink      91783
Inválido    52190
Name: count, dtype: int64

Com esse processamento vemos que os canais de uplink estão problemáticos, com mais de 1 terço deles fora de um bloco válido de frequência conforme determinado pela legislação. Os dados de Downlink estão muito mais amigáveis. Os canais de uplink foram definidos à partir da frequência de recepção cadastrada juntamente com designação licenciada para aquela estação. 

A abordagem a seguir será refazer a análise anterior para o uplink à partir da frequência de recepção teórica à partir do offset do dado bloco no qual se encaixa a frequência de transmissão. Assim os perfil de validade dos dados de Uplink terá a mesma estatística do Downlink. 

In [ ]:
df_out.to_parquet(folder / 'dados_reais_smp_agregados.parquet.gzip', index=False, compression='gzip')